In [48]:
## 

import pandas as pd
import numpy as np



In [49]:
## load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data
data.head()
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)


from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split


label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])

data.head()

from sklearn.preprocessing import OneHotEncoder

oneHotencoder_geo = OneHotEncoder()

geo_encoder = oneHotencoder_geo.fit_transform(data[["Geography"]])

geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [50]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=oneHotencoder_geo.get_feature_names_out(["Geography"]))
geo_encoder_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [51]:
## concat

df = pd.concat([data.drop("Geography",axis=1),geo_encoder_df],axis=1)

In [52]:
## independent and dependent data

X = df.drop("Exited",axis=1)
y = df["Exited"]

In [53]:
## split the dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.22,random_state=42)


In [54]:
## standararize

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [55]:
X_train

array([[-0.19485048, -1.09642404,  0.0094513 , ..., -0.9969278 ,
        -0.57912674,  1.73027616],
       [-0.66249484, -1.09642404,  0.29518832, ..., -0.9969278 ,
         1.72673776, -0.57794242],
       [ 0.55338049, -1.09642404, -0.18104004, ...,  1.00308167,
        -0.57912674, -0.57794242],
       ...,
       [ 0.8651434 , -1.09642404, -0.08579437, ...,  1.00308167,
        -0.57912674, -0.57794242],
       [ 0.15848081,  0.91205588,  0.39043399, ...,  1.00308167,
        -0.57912674, -0.57794242],
       [ 0.47024372,  0.91205588,  1.15239936, ..., -0.9969278 ,
         1.72673776, -0.57794242]], shape=(7800, 12))

In [56]:
y_train

5916    0
5129    0
8334    1
1918    0
8330    0
       ..
5734    0
5191    0
5390    1
860     1
7270    0
Name: Exited, Length: 7800, dtype: int64

In [57]:
import pickle

In [58]:
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("oneHotencoder_geo.pkl","wb") as file:
    pickle.dump(oneHotencoder_geo,file)
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)    

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### HyperparameterTunnig

In [ ]:
## Define a function to create the model and try different parameters(KerasClassifier)
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation="relu",input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation="relu"))

    model.add(Dense(1,activation="sigmoid"))

    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])  
    return model  

In [ ]:
! pip install scikeras

In [62]:
from scikeras.wrappers import KerasClassifier

In [69]:
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(
    model=create_model,   
    verbose=1
)


In [70]:
## define the grid search parametrs

param_grid = {
    "model__neurons": [16, 32, 64, 128],
    "model__layers": [1, 2],
    "epochs": [50, 100],
    "batch_size": [32]
}


In [71]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    n_jobs=-1,
    cv=3,
    verbose=1,
    error_score="raise"
)

grid_result = grid.fit(X_train, y_train)

print("Best score:", grid_result.best_score_)
print("Best params:", grid_result.best_params_)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


c:\Users\vipin\OneDrive\Desktop\DL\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7940 - loss: 0.4877
Epoch 2/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8103 - loss: 0.4380
Epoch 3/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8162 - loss: 0.4190
Epoch 4/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8262 - loss: 0.4045
Epoch 5/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8356 - loss: 0.3914
Epoch 6/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8392 - loss: 0.3799
Epoch 7/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8496 - loss: 0.3711
Epoch 8/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8506 - loss: 0.3643
Epoch 9/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8529 - loss: 0.3591
Epoch 10/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8544 - loss: 0.3555
Epoch 11/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8550 - loss: 0.3525
Epoch 12/50
244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step